In [2]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold, RepeatedKFold, KFold, RepeatedKFold, train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from joblib import dump, load

In [3]:
torch.cuda.is_available()

True

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [5]:
class MyDataset(Dataset):
    def __init__(self):
        super(MyDataset, self).__init__()
        df_left = pd.read_csv('./Results/INITIAL_SPLIT/left_train_modifiedMedPIQR.csv', index_col=0)
        df_right = pd.read_csv('./Results/INITIAL_SPLIT/right_train_modifiedMedPIQR.csv', index_col=0)

In [6]:
df_left = pd.read_csv('./Results/INITIAL_SPLIT/left_train_modifiedMedPIQR.csv', index_col=0)
df_right = pd.read_csv('./Results/INITIAL_SPLIT/right_train_modifiedMedPIQR.csv', index_col=0)
(df_left-df_right).sum().sum()

0.0

In [6]:
sf_10_0= ["area_lbankssts_medMIQR", "area_lbankssts_medPIQR", "area_lcaudalmiddlefrontal_medMIQR", "area_lcaudalmiddlefrontal_medPIQR", "area_lcuneus_medMIQR", "area_lcuneus_medPIQR", "area_lfusiform_medMIQR", "area_lfusiform_medPIQR", "area_linferiortemporal_medPIQR", "area_listhmuscingulate_medMIQR", "area_llateraloccipital_medMIQR", "area_llateralorbitofrontal_medMIQR", "area_llateralorbitofrontal_medPIQR", "area_llingual_medPIQR", "area_lmiddletemporal_medPIQR", "area_lparacentral_medMIQR", "area_lparsopercularis_medMIQR", "area_lparsopercularis_medPIQR", "area_lparstriangularis_medMIQR", "area_lparstriangularis_medPIQR", "area_lpostcentral_medMIQR", "area_lposteriorcingulate_medMIQR", "area_lprecentral_medMIQR", "area_lprecentral_medPIQR", "area_lprecuneus_medMIQR", "area_lrostralanteriorcingulate_medMIQR", "area_lrostralanteriorcingulate_medPIQR", "area_lsuperiorfrontal_medPIQR", "area_lsuperiorparietal_medMIQR", "area_lsuperiorparietal_medPIQR", "area_lsuperiortemporal_medMIQR", "area_lsuperiortemporal_medPIQR", "area_lsupramarginal_medMIQR", "area_lfrontalpole_medMIQR", "area_lfrontalpole_medPIQR", "area_ltemporalpole_medMIQR", "area_ltransversetemporal_medMIQR", "area_ltransversetemporal_medPIQR", "curv_lbankssts_medPIQR", "curv_lcaudalanteriorcingulate_medMIQR", "curv_lcaudalanteriorcingulate_medPIQR", "curv_lcaudalmiddlefrontal_medMIQR", "curv_lcuneus_medMIQR", "curv_lcuneus_medPIQR", "curv_lentorhinal_medMIQR", "curv_lentorhinal_medPIQR", "curv_lfusiform_medPIQR", "curv_linferiorparietal_medPIQR", "curv_linferiortemporal_medPIQR", "curv_listhmuscingulate_medMIQR", "curv_listhmuscingulate_medPIQR", "curv_llateraloccipital_medPIQR", "curv_lmedialorbitofrontal_medPIQR", "curv_lmiddletemporal_medMIQR", "curv_lmiddletemporal_medPIQR", "curv_lparacentral_medMIQR", "curv_lparacentral_medPIQR", "curv_lparsopercularis_medMIQR", "curv_lparsorbitalis_medMIQR", "curv_lparstriangularis_medMIQR", "curv_lposteriorcingulate_medMIQR", "curv_lposteriorcingulate_medPIQR", "curv_lprecentral_medMIQR", "curv_lprecentral_medPIQR", "curv_lprecuneus_medMIQR", "curv_lprecuneus_medPIQR", "curv_lsuperiorparietal_medPIQR", "curv_lsuperiortemporal_medPIQR", "curv_lsupramarginal_medMIQR", "curv_lfrontalpole_medMIQR", "curv_lfrontalpole_medPIQR", "curv_ltemporalpole_medMIQR", "curv_ltransversetemporal_medMIQR", "thickness_lbankssts_medMIQR", "thickness_lcaudalanteriorcingulate_medPIQR", "thickness_lcaudalmiddlefrontal_medMIQR", "thickness_lcuneus_medMIQR", "thickness_lcuneus_medPIQR", "thickness_lentorhinal_medMIQR", "thickness_linferiorparietal_medMIQR", "thickness_listhmuscingulate_medMIQR", "thickness_listhmuscingulate_medPIQR", "thickness_llateralorbitofrontal_medMIQR", "thickness_lmiddletemporal_medMIQR", "thickness_lmiddletemporal_medPIQR", "thickness_lparahippocampal_medMIQR", "thickness_lparahippocampal_medPIQR", "thickness_lparsopercularis_medMIQR", "thickness_lparstriangularis_medPIQR", "thickness_lpericalcarine_medMIQR", "thickness_lpericalcarine_medPIQR", "thickness_lpostcentral_medPIQR", "thickness_lposteriorcingulate_medPIQR", "thickness_lprecentral_medMIQR", "thickness_lprecentral_medPIQR", "thickness_lprecuneus_medPIQR", "thickness_lrostralanteriorcingulate_medPIQR", "thickness_lrostralmiddlefrontal_medPIQR", "thickness_lsuperiortemporal_medPIQR", "thickness_lsupramarginal_medMIQR", "thickness_lsupramarginal_medPIQR", "thickness_lfrontalpole_medMIQR", "thickness_ltemporalpole_medMIQR", "thickness_ltransversetemporal_medMIQR", "thickness_linsula_medPIQR", "volume_lcaudalanteriorcingulate_medMIQR", "volume_lcaudalanteriorcingulate_medPIQR", "volume_lcaudalmiddlefrontal_medPIQR", "volume_lcuneus_medMIQR", "volume_lcuneus_medPIQR", "volume_linferiorparietal_medPIQR", "volume_linferiortemporal_medMIQR", "volume_linferiortemporal_medPIQR", "volume_listhmuscingulate_medMIQR", "volume_listhmuscingulate_medPIQR", "volume_llateralorbitofrontal_medMIQR", "volume_llateralorbitofrontal_medPIQR", "volume_lmiddletemporal_medPIQR", "volume_lparahippocampal_medMIQR", "volume_lparacentral_medMIQR", "volume_lpericalcarine_medPIQR", "volume_lpostcentral_medMIQR", "volume_lpostcentral_medPIQR", "volume_lposteriorcingulate_medMIQR", "volume_lprecuneus_medMIQR", "volume_lprecuneus_medPIQR", "volume_lrostralanteriorcingulate_medMIQR", "volume_lrostralanteriorcingulate_medPIQR", "volume_lrostralmiddlefrontal_medMIQR", "volume_lrostralmiddlefrontal_medPIQR", "volume_lsupramarginal_medPIQR", "volume_lfrontalpole_medMIQR", "volume_lfrontalpole_medPIQR", "volume_ltemporalpole_medMIQR", "volume_ltransversetemporal_medMIQR", "volume_ltransversetemporal_medPIQR"]
sf_14_0 = ["area_lcaudalanteriorcingulate_medMIQR", "area_lcaudalmiddlefrontal_medPIQR", "area_lcuneus_medPIQR", "area_lentorhinal_medMIQR", "area_lfusiform_medPIQR", "area_linferiorparietal_medMIQR", "area_listhmuscingulate_medPIQR", "area_llateralorbitofrontal_medMIQR", "area_lmedialorbitofrontal_medMIQR", "area_lmedialorbitofrontal_medPIQR", "area_lmiddletemporal_medPIQR", "area_lparahippocampal_medMIQR", "area_lparacentral_medMIQR", "area_lparacentral_medPIQR", "area_lparsopercularis_medMIQR", "area_lparsopercularis_medPIQR", "area_lparsorbitalis_medMIQR", "area_lparstriangularis_medMIQR", "area_lparstriangularis_medPIQR", "area_lposteriorcingulate_medMIQR", "area_lposteriorcingulate_medPIQR", "area_lprecentral_medMIQR", "area_lprecuneus_medMIQR", "area_lprecuneus_medPIQR", "area_lrostralanteriorcingulate_medPIQR", "area_lrostralmiddlefrontal_medPIQR", "area_lsuperiorfrontal_medMIQR", "area_lsuperiortemporal_medPIQR", "area_lsupramarginal_medMIQR", "area_lsupramarginal_medPIQR", "area_ltemporalpole_medMIQR", "area_ltemporalpole_medPIQR", "area_linsula_medMIQR", "curv_lbankssts_medPIQR", "curv_lcaudalanteriorcingulate_medPIQR", "curv_lcuneus_medMIQR", "curv_lcuneus_medPIQR", "curv_lfusiform_medMIQR", "curv_lfusiform_medPIQR", "curv_linferiorparietal_medMIQR", "curv_linferiortemporal_medMIQR", "curv_linferiortemporal_medPIQR", "curv_listhmuscingulate_medMIQR", "curv_llateraloccipital_medMIQR", "curv_llateraloccipital_medPIQR", "curv_llateralorbitofrontal_medMIQR", "curv_llateralorbitofrontal_medPIQR", "curv_llingual_medMIQR", "curv_llingual_medPIQR", "curv_lmiddletemporal_medMIQR", "curv_lmiddletemporal_medPIQR", "curv_lparacentral_medPIQR", "curv_lparstriangularis_medMIQR", "curv_lparstriangularis_medPIQR", "curv_lpostcentral_medMIQR", "curv_lposteriorcingulate_medMIQR", "curv_lprecentral_medMIQR", "curv_lprecentral_medPIQR", "curv_lprecuneus_medMIQR", "curv_lprecuneus_medPIQR", "curv_lrostralanteriorcingulate_medMIQR", "curv_lrostralmiddlefrontal_medMIQR", "curv_lsuperiorfrontal_medMIQR", "curv_lsuperiorfrontal_medPIQR", "curv_lsuperiorparietal_medPIQR", "curv_lsupramarginal_medMIQR", "curv_lsupramarginal_medPIQR", "curv_ltemporalpole_medMIQR", "curv_ltemporalpole_medPIQR", "curv_ltransversetemporal_medMIQR", "curv_ltransversetemporal_medPIQR", "curv_linsula_medPIQR", "thickness_lbankssts_medPIQR", "thickness_lcaudalanteriorcingulate_medPIQR", "thickness_lcuneus_medMIQR", "thickness_lcuneus_medPIQR", "thickness_lentorhinal_medMIQR", "thickness_lentorhinal_medPIQR", "thickness_lfusiform_medPIQR", "thickness_linferiorparietal_medPIQR", "thickness_llateralorbitofrontal_medMIQR", "thickness_llateralorbitofrontal_medPIQR", "thickness_lmedialorbitofrontal_medMIQR", "thickness_lmedialorbitofrontal_medPIQR", "thickness_lmiddletemporal_medMIQR", "thickness_lmiddletemporal_medPIQR", "thickness_lparahippocampal_medMIQR", "thickness_lparahippocampal_medPIQR", "thickness_lparacentral_medMIQR", "thickness_lparstriangularis_medMIQR", "thickness_lpericalcarine_medPIQR", "thickness_lpostcentral_medMIQR", "thickness_lpostcentral_medPIQR", "thickness_lposteriorcingulate_medMIQR", "thickness_lprecentral_medPIQR", "thickness_lprecuneus_medMIQR", "thickness_lprecuneus_medPIQR", "thickness_lrostralanteriorcingulate_medPIQR", "thickness_lrostralmiddlefrontal_medPIQR", "thickness_lsuperiorparietal_medPIQR", "thickness_lsuperiortemporal_medPIQR", "thickness_lsupramarginal_medPIQR", "thickness_lfrontalpole_medMIQR", "thickness_lfrontalpole_medPIQR", "thickness_ltransversetemporal_medMIQR", "thickness_ltransversetemporal_medPIQR", "thickness_linsula_medMIQR", "volume_lbankssts_medMIQR", "volume_lcaudalanteriorcingulate_medMIQR", "volume_lcaudalmiddlefrontal_medMIQR", "volume_lentorhinal_medPIQR", "volume_lfusiform_medMIQR", "volume_lfusiform_medPIQR", "volume_linferiorparietal_medMIQR", "volume_linferiortemporal_medPIQR", "volume_llateraloccipital_medMIQR", "volume_llateraloccipital_medPIQR", "volume_llateralorbitofrontal_medMIQR", "volume_llateralorbitofrontal_medPIQR", "volume_lmedialorbitofrontal_medPIQR", "volume_lmiddletemporal_medPIQR", "volume_lparahippocampal_medPIQR", "volume_lparsopercularis_medMIQR", "volume_lparsorbitalis_medMIQR", "volume_lparsorbitalis_medPIQR", "volume_lpericalcarine_medPIQR", "volume_lposteriorcingulate_medPIQR", "volume_lprecentral_medMIQR", "volume_lprecuneus_medMIQR", "volume_lprecuneus_medPIQR", "volume_lrostralanteriorcingulate_medMIQR", "volume_lsuperiortemporal_medPIQR", "volume_lsupramarginal_medMIQR", "volume_ltemporalpole_medMIQR", "volume_ltemporalpole_medPIQR", "volume_ltransversetemporal_medMIQR"]

In [7]:
len(sf_10_0)

136

In [8]:
len(sf_14_0)

136

In [18]:
cnt = 0
for feat in sf_10_0:
    if feat not in sf_14_0:
        print(feat)
        cnt+=1

area_lbankssts_medMIQR
area_lbankssts_medPIQR
area_lcaudalmiddlefrontal_medMIQR
area_lcuneus_medMIQR
area_lfusiform_medMIQR
area_linferiortemporal_medPIQR
area_listhmuscingulate_medMIQR
area_llateraloccipital_medMIQR
area_llateralorbitofrontal_medPIQR
area_llingual_medPIQR
area_lpostcentral_medMIQR
area_lprecentral_medPIQR
area_lrostralanteriorcingulate_medMIQR
area_lsuperiorfrontal_medPIQR
area_lsuperiorparietal_medMIQR
area_lsuperiorparietal_medPIQR
area_lsuperiortemporal_medMIQR
area_lfrontalpole_medMIQR
area_lfrontalpole_medPIQR
area_ltransversetemporal_medMIQR
area_ltransversetemporal_medPIQR
curv_lcaudalanteriorcingulate_medMIQR
curv_lcaudalmiddlefrontal_medMIQR
curv_lentorhinal_medMIQR
curv_lentorhinal_medPIQR
curv_linferiorparietal_medPIQR
curv_listhmuscingulate_medPIQR
curv_lmedialorbitofrontal_medPIQR
curv_lparacentral_medMIQR
curv_lparsopercularis_medMIQR
curv_lparsorbitalis_medMIQR
curv_lposteriorcingulate_medPIQR
curv_lsuperiortemporal_medPIQR
curv_lfrontalpole_medMIQR
cur

In [19]:
cnt

68

In [17]:
area_lcaudalanteriorcingulate_medMIQR
area_lentorhinal_medMIQR
area_linferiorparietal_medMIQR
area_listhmuscingulate_medPIQR
area_lmedialorbitofrontal_medMIQR
area_lmedialorbitofrontal_medPIQR
area_lparahippocampal_medMIQR
area_lparacentral_medPIQR
area_lparsorbitalis_medMIQR
area_lposteriorcingulate_medPIQR
area_lprecuneus_medPIQR
area_lrostralmiddlefrontal_medPIQR
area_lsuperiorfrontal_medMIQR
area_lsupramarginal_medPIQR
area_ltemporalpole_medPIQR
area_linsula_medMIQR
curv_lfusiform_medMIQR
curv_linferiorparietal_medMIQR
curv_linferiortemporal_medMIQR
curv_llateraloccipital_medMIQR
curv_llateralorbitofrontal_medMIQR
curv_llateralorbitofrontal_medPIQR
curv_llingual_medMIQR
curv_llingual_medPIQR
curv_lparstriangularis_medPIQR
curv_lpostcentral_medMIQR
curv_lrostralanteriorcingulate_medMIQR
curv_lrostralmiddlefrontal_medMIQR
curv_lsuperiorfrontal_medMIQR
curv_lsuperiorfrontal_medPIQR
curv_lsupramarginal_medPIQR
curv_ltemporalpole_medPIQR
curv_ltransversetemporal_medPIQR
curv_linsula_medPIQR
thickness_lbankssts_medPIQR
thickness_lentorhinal_medPIQR
thickness_lfusiform_medPIQR
thickness_linferiorparietal_medPIQR
thickness_llateralorbitofrontal_medPIQR
thickness_lmedialorbitofrontal_medMIQR
thickness_lmedialorbitofrontal_medPIQR
thickness_lparacentral_medMIQR
thickness_lparstriangularis_medMIQR
thickness_lpostcentral_medMIQR
thickness_lposteriorcingulate_medMIQR
thickness_lprecuneus_medMIQR
thickness_lsuperiorparietal_medPIQR
thickness_lfrontalpole_medPIQR
thickness_ltransversetemporal_medPIQR
thickness_linsula_medMIQR
volume_lbankssts_medMIQR
volume_lcaudalmiddlefrontal_medMIQR
volume_lentorhinal_medPIQR
volume_lfusiform_medMIQR
volume_lfusiform_medPIQR
volume_linferiorparietal_medMIQR
volume_llateraloccipital_medMIQR
volume_llateraloccipital_medPIQR
volume_lmedialorbitofrontal_medPIQR
volume_lparahippocampal_medPIQR
volume_lparsopercularis_medMIQR
volume_lparsorbitalis_medMIQR
volume_lparsorbitalis_medPIQR
volume_lposteriorcingulate_medPIQR
volume_lprecentral_medMIQR
volume_lsuperiortemporal_medPIQR
volume_lsupramarginal_medMIQR
volume_ltemporalpole_medPIQR

NameError: name 'area_lcaudalanteriorcingulate_medMIQR' is not defined

In [ ]:
area_lbankssts_medMIQR
area_lbankssts_medPIQR
area_lcaudalmiddlefrontal_medMIQR
area_lcuneus_medMIQR
area_lfusiform_medMIQR
area_linferiortemporal_medPIQR
area_listhmuscingulate_medMIQR
area_llateraloccipital_medMIQR
area_llateralorbitofrontal_medPIQR
area_llingual_medPIQR
area_lpostcentral_medMIQR
area_lprecentral_medPIQR
area_lrostralanteriorcingulate_medMIQR
area_lsuperiorfrontal_medPIQR
area_lsuperiorparietal_medMIQR
area_lsuperiorparietal_medPIQR
area_lsuperiortemporal_medMIQR
area_lfrontalpole_medMIQR
area_lfrontalpole_medPIQR
area_ltransversetemporal_medMIQR
area_ltransversetemporal_medPIQR
curv_lcaudalanteriorcingulate_medMIQR
curv_lcaudalmiddlefrontal_medMIQR
curv_lentorhinal_medMIQR
curv_lentorhinal_medPIQR
curv_linferiorparietal_medPIQR
curv_listhmuscingulate_medPIQR
curv_lmedialorbitofrontal_medPIQR
curv_lparacentral_medMIQR
curv_lparsopercularis_medMIQR
curv_lparsorbitalis_medMIQR
curv_lposteriorcingulate_medPIQR
curv_lsuperiortemporal_medPIQR
curv_lfrontalpole_medMIQR
curv_lfrontalpole_medPIQR
thickness_lbankssts_medMIQR
thickness_lcaudalmiddlefrontal_medMIQR
thickness_linferiorparietal_medMIQR
thickness_listhmuscingulate_medMIQR
thickness_listhmuscingulate_medPIQR
thickness_lparsopercularis_medMIQR
thickness_lparstriangularis_medPIQR
thickness_lpericalcarine_medMIQR
thickness_lposteriorcingulate_medPIQR
thickness_lprecentral_medMIQR
thickness_lsupramarginal_medMIQR
thickness_ltemporalpole_medMIQR
thickness_linsula_medPIQR
volume_lcaudalanteriorcingulate_medPIQR
volume_lcaudalmiddlefrontal_medPIQR
volume_lcuneus_medMIQR
volume_lcuneus_medPIQR
volume_linferiorparietal_medPIQR
volume_linferiortemporal_medMIQR
volume_listhmuscingulate_medMIQR
volume_listhmuscingulate_medPIQR
volume_lparahippocampal_medMIQR
volume_lparacentral_medMIQR
volume_lpostcentral_medMIQR
volume_lpostcentral_medPIQR
volume_lposteriorcingulate_medMIQR
volume_lrostralanteriorcingulate_medPIQR
volume_lrostralmiddlefrontal_medMIQR
volume_lrostralmiddlefrontal_medPIQR
volume_lsupramarginal_medPIQR
volume_lfrontalpole_medMIQR
volume_lfrontalpole_medPIQR
volume_ltransversetemporal_medPIQR